### Combination of Test 6 & 7
AI agent with memory

In [ ]:
import os
import sys
from dotenv import load_dotenv

# LangChain core components
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Message history storage
from langchain_community.chat_message_histories.upstash_redis import UpstashRedisChatMessageHistory

# Document processing
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vector stores
from langchain_community.vectorstores.faiss import FAISS

# Agents and tools
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool


current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_8.ipynb')
git_add(file_path)
git_commit('Updated test 8')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

Error executing command: ['git', 'commit', '-m', 'Updated test 8']
Error message: 


KeyboardInterrupt: 

Login to https://upstash.com/?utm_source=Leon_LangchainPython
to save the chat history

In [ ]:
! pip install upstash_redis

In [ ]:
def load_and_process_document(url, chunk_size=1000, chunk_overlap=200):
    """Load a document from the web, split it into chunks, generate embeddings, and store them in a FAISS index."""
    loader = WebBaseLoader(url)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = splitter.split_documents(docs)
    embedding = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(split_docs, embedding=embedding)
    return vector_store.as_retriever(search_kwargs={"k": 5}), docs

def initialize_chat_model():
    """Initialize the chat model with a prompt template."""
    return ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

def create_prompt(context):
    """Create a structured prompt for the AI agent."""
    return ChatPromptTemplate.from_messages([
        ("system", f"You are a friendly AI assistant. Answer questions based on the context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

def initialize_memory():
    """Set up chat history storage with Upstash Redis."""
    history = UpstashRedisChatMessageHistory(
        url=os.getenv("UPSTASH_REDIS_REST_URL"),
        token=os.getenv("UPSTASH_REDIS_REST_TOKEN"),
        session_id="chat_1",
        ttl=0  # Conversation will never expire
    )
    return ConversationBufferMemory(memory_key="chat_history", return_messages=True, chat_memory=history)

def create_agent(retriever, model, prompt, memory):
    """Create an AI agent with tools for document retrieval and web search."""
    retriever_tool = create_retriever_tool(retriever, "toolbox_search", "Use this tool when searching for information about FreeSurfer")
    search_tool = TavilySearchResults()
    tools = [search_tool, retriever_tool]
    agent = create_openai_functions_agent(llm=model, prompt=prompt, tools=tools)
    return AgentExecutor(agent=agent, tools=tools, memory=memory)

def process_chat(agent_executor, user_input):
    """Process a chat query using the AI agent."""
    response = agent_executor.invoke({"input": user_input})
    return response["output"]

def main():
    """Main function to run the chat interface."""
    retriever, docs = load_and_process_document("https://en.wikipedia.org/wiki/FreeSurfer")
    model = initialize_chat_model()
    
    # Extract text from the loaded documents
    context_text = " ".join([doc.page_content for doc in docs])
    
    prompt = create_prompt(context_text)  # Pass extracted text as context
    memory = initialize_memory()
    agent_executor = create_agent(retriever, model, prompt, memory)
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = process_chat(agent_executor, user_input)
        print("AI:", response)


if __name__ == "__main__":
    main()
